In [ ]:
from glob import glob
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import os
########################################################################
#glob is use to list all the file in a directory
########################################################################
from glob import glob
########################################################################
#The main package to work with audio data
########################################################################
import librosa
import librosa.display

#to listen to sound
import IPython.display as ipd
from itertools import cycle

from sklearn import linear_model
import sklearn.metrics
from sklearn.preprocessing import scale, normalize

In [ ]:
#For color and make my slide look nice

sns.set_theme(style='white', palette=None)
color_pal=plt.rcParams["axes.prop_cycle"].by_key()['color']
color_cycle=cycle(plt.rcParams["axes.prop_cycle"].by_key()['color'])

In [ ]:
path="/content/drive/MyDrive/DATASETS/TESS/"

anger = glob(path+"Anger/*.wav")
bored = glob(path+"Bored/*.wav")
disgust = glob(path+"Disgust/*.wav")
fear = glob(path+"Fear/*.wav")
happy = glob(path+"Happy/*.wav")
#neutral=glob(path+"Neutral/*.wav")
sad = glob(path+"Sad/*.wav")


In [ ]:
!pip install resampy
#!apt install python3-resampy
import resampy

In [ ]:
def mfccs_features(audio_files):
  extracted_features = []
  label = [1,2,3,4,5]
  for audio_file in audio_files:

    if audio_file== anger:
      x = 0
    elif audio_file == happy:
      x = 1
    elif audio_file == fear:
      x = 2
    elif audio_file == sad:
      x = 3
    else:
      x = 4

    for i in range(len(audio_file)):
      audio, sample_rate = librosa.load(audio_file[i], res_type='kaiser_best')
      mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
      mfccs_features_scale = np.mean(mfccs_features.T, axis=0)
      class_label = label[x]
      data = mfccs_features_scale
      extracted_features.append([data,class_label])
  return extracted_features


In [ ]:
extracted_features = mfccs_features([anger,happy,fear,sad,disgust] )


In [ ]:
#We create a data frame containg our mfc coefficient as feature and act or dog as label
Audio_data=pd.DataFrame(extracted_features, columns=['Features', 'Class'])
#Audio_data


#We randomly shuffle our dataframe
data = Audio_data.sample(frac=1, random_state=1).reset_index()
data.drop('index', axis=1, inplace=True)
data



,Features,Class
0,"[-315.57455, 18.927155, -17.957874, 17.602816,...",2
1,"[-459.61826, 81.57872, 22.036575, 15.578322, 1...",5
2,"[-520.4158, 73.12935, 22.896946, 14.846818, 15...",4
3,"[-509.79453, 97.24656, 26.216064, 16.730312, 2...",4
4,"[-478.13864, 83.59039, 13.006593, 8.810062, 7....",4
...,...,...
1995,"[-445.62787, 65.73753, 13.454742, 0.32482845, ...",5
1996,"[-314.14777, 44.58899, -23.835648, 20.176666, ...",3
1997,"[-428.83334, 90.23989, -2.3602102, 22.070618, ...",5
1998,"[-358.72388, 21.980139, -3.0597188, 30.210419,...",1


In [ ]:
#We extract our feature matrix and we normalize it
X_matrix=normalize(np.array(data['Features'].tolist()))

#we extract our response variable containig the class of each audio feature
y_vector=np.array(data['Class'].tolist())

In [ ]:

#We look at the shape of our feature matrix
X_matrix.shape

(2000, 40)

In [ ]:
#We split in training and testing set with 70% for training and 30% for testing
from sklearn.model_selection import train_test_split
X_train, X_test,Y_train,Y_test=train_test_split(X_matrix,y_vector, test_size=0.3, random_state=0)


In [ ]:
#We create an instance of our support vector machine with a linear kernel and a regularizer C=3000
from sklearn.svm import SVC
model = SVC(kernel='linear', C=6000)

In [ ]:
#We fit our model using de training set
model.fit(X_train,Y_train)

SVC(C=6000, kernel='linear')

In [ ]:
#We then compute the predicted value of our model using the trainig and testing set
y_pred_train  = model.predict(X_train)
y_pred_test = model.predict(X_test)

In [ ]:
#We load one of the cat sound and we check the prediction outcome
#aud1=glob("C:/Users/User/Desktop/AIMS2022-23/REVIEW BLOCK/BLOCK 3/DATA SCIENCE/GROUP ASSIGNMENT/cats_dogs/train/cat/*.wav")
ipd.Audio(anger[0])

In [ ]:
#We predict wether is a cat or dog audio
audio, sample_rate=librosa.load(anger[2], res_type='kaiser_best')
mfccs_features= librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_features_scale=np.mean(mfccs_features.T, axis=0)
x2=normalize(np.array(mfccs_features_scale.tolist()).reshape(1,40))
y=model.predict(x2)
y[0]

1

In [ ]:
#We compute the accuracy score of our model
import sklearn.metrics
#Training accuracy
sklearn.metrics.accuracy_score(y_pred_train , Y_train)

1.0

In [ ]:
#Testing accuracy
sklearn.metrics.accuracy_score(y_pred_test , Y_test)

0.9966666666666667

In [ ]:
#Confusion matrix for training
sklearn.metrics.confusion_matrix(Y_train,y_pred_train)

array([[275,   0,   0,   0,   0],
       [  0, 279,   0,   0,   0],
       [  0,   0, 292,   0,   0],
       [  0,   0,   0, 289,   0],
       [  0,   0,   0,   0, 265]])

In [ ]:
#Confusion matrix for testing
sklearn.metrics.confusion_matrix(Y_test,y_pred_test)

array([[125,   0,   0,   0,   0],
       [  1, 120,   0,   0,   0],
       [  0,   0, 108,   0,   0],
       [  0,   0,   0, 111,   0],
       [  0,   1,   0,   0, 134]])

In [ ]:
#F1 score for training
sklearn.metrics.f1_score(Y_train,y_pred_train,average="micro")

1.0

In [ ]:
#F1 score for testing
sklearn.metrics.f1_score(Y_test,y_pred_test,average="micro")

0.9966666666666667

In [ ]:
#Recall score for training
sklearn.metrics.recall_score(Y_train,y_pred_train,average="micro")

1.0

In [ ]:
#Recall score for testing
sklearn.metrics.recall_score(Y_test,y_pred_test,average="micro")

0.9966666666666667

In [ ]:
#Precision score for training
sklearn.metrics.precision_score(Y_train,y_pred_train,average="micro")

1.0

In [ ]:
#Precision score for testing
sklearn.metrics.precision_score(Y_test,y_pred_test,average="micro")

0.9966666666666667

In [ ]:
#We split in training and testing set with 70% for trainig and 30% for testing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, Flatten, Dense,GlobalAveragePooling2D, Dropout
from tensorflow.keras import metrics
#from keras.utils import np_utils
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import plot_model,to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

le = LabelEncoder()
y_vector1 = to_categorical(le.fit_transform(y_vector))
X_train_NN, X_test_NN,Y_train_NN,Y_test_NN=train_test_split(X_matrix,y_vector1, test_size=0.3, random_state=0)

In [ ]:
def network():

    # Seeting a seed so we can reproduce the results
    tf.random.set_seed(137)

    # Define the initialisation method
    initializer = RandomNormal(mean = 0.0, stddev=0.01)

    # create a squential model
    model = Sequential()

    # add fully connected layer
    model.add(Dense(40, input_dim=40, activation='relu', kernel_initializer = initializer))
    model.add(Dense(256, activation='relu', kernel_initializer = initializer))
    model.add(Dropout(0.4))
    model.add(Dense(512, activation='relu', kernel_initializer = initializer))
    model.add(Dropout(0.4))

    model.add(Dense(128, activation='relu', kernel_initializer = initializer))
    model.add(Dropout(0.4))
    # add one fully connected output layer with a softmax activation
    model.add(Dense(5, activation='softmax', kernel_initializer = initializer))

    # Define the optimiser
    optimiser = Adam(learning_rate=0.001)

    # Define the loss function
    loss = CategoricalCrossentropy()

    # Compile model
    model.compile(loss=loss,
                  optimizer=optimiser,
                  metrics=['accuracy'])

    return model

In [ ]:
model1=network()
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 40)                1640      
                                                                 
 dense_1 (Dense)             (None, 256)               10496     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 512)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               65664     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [ ]:
earlystop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')
history1 = model1.fit(X_train_NN, Y_train_NN,epochs=300)

Epoch 1/300
44/44 [==============================] - 2s 8ms/step - loss: 1.6097 - accuracy: 0.1943
Epoch 2/300
44/44 [==============================] - 0s 8ms/step - loss: 1.6094 - accuracy: 0.1886
Epoch 3/300
44/44 [==============================] - 0s 7ms/step - loss: 1.6093 - accuracy: 0.2086
Epoch 4/300
44/44 [==============================] - 0s 6ms/step - loss: 1.6091 - accuracy: 0.2086
Epoch 5/300
44/44 [==============================] - 0s 7ms/step - loss: 1.4727 - accuracy: 0.2893
Epoch 6/300
44/44 [==============================] - 0s 7ms/step - loss: 1.2078 - accuracy: 0.4186
Epoch 7/300
44/44 [==============================] - 0s 7ms/step - loss: 1.0622 - accuracy: 0.4750
Epoch 8/300
44/44 [==============================] - 0s 8ms/step - loss: 0.9391 - accuracy: 0.5614
Epoch 9/300
44/44 [==============================] - 0s 7ms/step - loss: 0.7688 - accuracy: 0.6414
Epoch 10/300
44/44 [==============================] - 0s 7ms/step - loss: 0.7187 - accuracy: 0.6464
Epoch 11/

In [ ]:
y_pred_train_NN = np.argmax(model1.predict(X_train_NN), axis=-1)
y_pred_test_NN = np.argmax(model1.predict(X_test_NN), axis=-1)

19/19 [==============================] - 0s 5ms/step


In [ ]:
sklearn.metrics.accuracy_score(np.argmax(Y_train_NN,axis=-1), y_pred_train_NN)

0.9914285714285714

In [ ]:
sklearn.metrics.accuracy_score(np.argmax(Y_test_NN,axis=-1), y_pred_test_NN)

0.9716666666666667

In [ ]:
sklearn.metrics.f1_score(np.argmax(Y_train_NN,axis=-1), y_pred_train_NN, average='micro')

0.9914285714285714

In [ ]:
sklearn.metrics.f1_score(np.argmax(Y_test_NN,axis=-1), y_pred_test_NN, average='micro')

0.9716666666666667

In [ ]:
sklearn.metrics.recall_score(np.argmax(Y_train_NN,axis=-1), y_pred_train_NN, average='micro')

0.9914285714285714

In [ ]:
sklearn.metrics.recall_score(np.argmax(Y_test_NN,axis=-1), y_pred_test_NN, average='micro')

0.9716666666666667

In [ ]:
sklearn.metrics.precision_score(np.argmax(Y_test_NN,axis=-1), y_pred_test_NN, average='micro')

0.9716666666666667

In [ ]:
sklearn.metrics.precision_score(np.argmax(Y_train_NN,axis=-1), y_pred_train_NN, average='micro')

0.9914285714285714

In [ ]:
#sklearn.metrics.confusion_matrix(Y_test_NN,y_pred_test_NN)

len(Y_test_NN)
len(y_pred_test_NN)

600